# 🎤 OIIAIOIIIAI 元音识别 CNN 训练 (Vowel Dataset)

本 notebook 使用 **Dataset of Vowels** 训练元音识别模型。

## 为什么换数据集？

之前使用 TIMIT 数据集的问题：
- TIMIT 中的元音被辅音包围（如 "bat", "bet"），存在**协同发音效应**
- 模型学到的是「辅音-元音-辅音」的过渡特征，而非纯元音特征
- 验证集准确率 92%，但在真实的纯元音输入（游戏场景）表现很差

## Dataset of Vowels 数据集

- 来源：Kaggle `darubiano57/dataset-of-vowels`
- 包含 **16875 个孤立元音录音** (a/e/i/o/u)
- 男性 (h) 和女性 (m) 说话者
- 文件名格式：`{性别}{元音}_{说话者ID}_ ({序号}).wav`

## 数据预处理

原始数据存在以下问题，需要预处理：
1. **长度不一**：50ms ~ 2s，需要检测有效发音区域
2. **延迟发音**：录音开始后才发音，需要用能量检测定位
3. **音量不一**：需要归一化
4. **环境噪声**：可作为数据增强的一部分

## 数据增强策略

1. 随机拼接 2-4 个元音片段，模拟 "OIIAIOIIIAI" 式的连续输入
2. 添加背景噪声（白噪声、粉噪声）
3. 应用音量变化、轻微时移
4. 从拼接序列中提取 210ms 窗口作为训练样本

## 模型规格
- **输入**: 210ms 原始波形 (3360 samples @ 16kHz)
- **输出**: 6 类 (A/E/I/O/U/silence)
- **格式**: TensorFlow.js，可直接在浏览器运行

## 1. 环境配置

In [ ]:
# 清理之前的训练目录（如需要）
!rm -rf /content/vowel_training

In [ ]:
# 挂载 Google Drive（用于保存输出）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 安装必要的包（仅执行一次）
import subprocess
import sys

def is_installed(pkg):
    try:
        __import__(pkg)
        return True
    except ImportError:
        return False

packages_to_check = ['tensorflowjs', 'soundfile', 'kagglehub']
need_install = not all(is_installed(pkg.replace('-', '_')) for pkg in packages_to_check)

if need_install:
    # 安装 kagglehub（新的 Kaggle 数据集下载方式）
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'kagglehub'], capture_output=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'soundfile', 'tensorflowjs'], capture_output=True)
    print("✅ 包安装完成")
else:
    print("✅ 包已安装，跳过")

In [ ]:
# 检查 GPU
!nvidia-smi

import tensorflow as tf
print(f"\nTensorFlow 版本: {tf.__version__}")
print(f"GPU 可用: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# 检查 GPU
!nvidia-smi

import tensorflow as tf
print(f"\nTensorFlow 版本: {tf.__version__}")
print(f"GPU 可用: {tf.config.list_physical_devices('GPU')}")

## 2. 下载 Dataset of Vowels

In [ ]:
import kagglehub
import os

# 下载数据集
print("正在下载 Dataset of Vowels...")
dataset_path = kagglehub.dataset_download("darubiano57/dataset-of-vowels")
print(f"✅ 数据集已下载到: {dataset_path}")

# 查找实际的数据目录
# 数据集结构：DATASET_OF_VOWELS 目录下有 wav 文件
vowel_data_dir = os.path.join(dataset_path, 'DATASET_OF_VOWELS')
if not os.path.exists(vowel_data_dir):
    # 如果没有子目录，可能直接在根目录
    vowel_data_dir = dataset_path

print(f"\n数据目录: {vowel_data_dir}")

# 统计文件
import glob
wav_files = glob.glob(os.path.join(vowel_data_dir, '*.wav'))
print(f"找到 {len(wav_files)} 个 wav 文件")

# 显示一些文件名示例
print("\n文件名示例:")
for f in wav_files[:10]:
    print(f"  {os.path.basename(f)}")

## 3. 配置参数

In [ ]:
import os
import numpy as np

# ============ 数据路径（由上一步自动设置） ============
VOWEL_DATA_DIR = vowel_data_dir
# =====================================================

# 工作目录
WORK_DIR = '/content/vowel_training'
DATA_DIR = os.path.join(WORK_DIR, 'data')
CHECKPOINT_DIR = os.path.join(WORK_DIR, 'checkpoints')
OUTPUT_DIR = os.path.join(WORK_DIR, 'output')

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 音频参数（与 TIMIT 版本保持一致）
SAMPLE_RATE = 16000           # 16kHz 采样率
FRAME_LENGTH = 400            # 25ms 帧长
FRAME_STEP = 160              # 10ms 帧移
FFT_LENGTH = 512              # FFT 点数
N_MELS = 40                   # Mel 频带数
INPUT_SAMPLES = 3360          # 210ms @ 16kHz

# 元音类别
VOWEL_CLASSES = ['A', 'E', 'I', 'O', 'U', 'silence']
NUM_CLASSES = len(VOWEL_CLASSES)

# 文件名格式: {性别}{元音}_{说话者ID}_ ({序号}).wav
# 例如: ho_1_ (80).wav = 男性(h) + O元音 + 说话者1 + 第80个样本
# 元音字母映射（小写 -> 大写）
VOWEL_LETTER_MAP = {
    'a': 'A',
    'e': 'E',
    'i': 'I',
    'o': 'O',
    'u': 'U',
}

# 训练配置
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001
EARLY_STOPPING_PATIENCE = 10

# 数据增强配置
AUGMENT_CONFIG = {
    'concat_min_vowels': 2,      # 最少拼接元音数
    'concat_max_vowels': 4,      # 最多拼接元音数
    'noise_prob': 0.5,           # 添加噪声的概率
    'noise_level_range': (0.001, 0.01),  # 噪声强度范围
    'volume_range': (0.7, 1.3),  # 音量变化范围
    'samples_per_vowel': 500,    # 每个元音生成的样本数
}

# 音频预处理配置（针对数据质量问题）
PREPROCESS_CONFIG = {
    'energy_threshold': 0.01,    # 能量阈值（用于检测发音开始/结束）
    'min_vowel_duration': 0.05,  # 最小有效元音时长 (50ms)
    'max_vowel_duration': 1.0,   # 最大截取时长 (1s)
    'onset_window_ms': 10,       # 能量检测窗口大小 (ms)
    'onset_hop_ms': 5,           # 能量检测步长 (ms)
}

print("✅ 配置完成!")
print(f"   数据路径: {VOWEL_DATA_DIR}")
print(f"   输出目录: {OUTPUT_DIR}")
print(f"   输入长度: {INPUT_SAMPLES} samples ({INPUT_SAMPLES/SAMPLE_RATE*1000:.0f}ms)")

## 4. 加载并预处理数据

关键预处理步骤：
1. **能量检测**：找到音频中实际发音的起始和结束位置
2. **有效区域截取**：只保留有声音的部分
3. **音量归一化**：统一音量水平

In [ ]:
import librosa
import soundfile as sf
from collections import defaultdict
from tqdm import tqdm
import re

def load_audio_file(filepath, target_sr=SAMPLE_RATE):
    """
    加载音频文件并重采样到目标采样率
    """
    try:
        audio, sr = librosa.load(filepath, sr=target_sr, mono=True)
        return audio
    except Exception as e:
        try:
            audio, sr = sf.read(filepath)
            if len(audio.shape) > 1:
                audio = audio.mean(axis=1)  # 转单声道
            if sr != target_sr:
                audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
            return audio.astype(np.float32)
        except Exception as e2:
            print(f"⚠️ 无法加载: {filepath}, 错误: {e2}")
            return None


def parse_vowel_filename(filename):
    """
    从文件名解析元音类型和说话者信息
    
    文件名格式: {性别}{元音}_{说话者ID}_ ({序号}).wav
    例如: ho_1_ (80).wav = 男性(h) + O元音 + 说话者1 + 第80个样本
          ma_2_ (19).wav = 女性(m) + A元音 + 说话者2 + 第19个样本
    
    Returns:
        vowel_class: 'A', 'E', 'I', 'O', 'U' 或 None
        speaker: 说话者标识 (如 'h1', 'm2')
        gender: 'male' 或 'female'
    """
    basename = os.path.splitext(filename)[0].lower()
    
    # 匹配模式: {h或m}{a/e/i/o/u}_{数字}_ ({数字})
    match = re.match(r'^([hm])([aeiou])_(\d+)_\s*\((\d+)\)$', basename)
    
    if not match:
        return None, None, None
    
    gender_code = match.group(1)
    vowel_letter = match.group(2)
    speaker_id = match.group(3)
    
    gender = 'male' if gender_code == 'h' else 'female'
    speaker = f"{gender_code}{speaker_id}"
    
    vowel_class = VOWEL_LETTER_MAP.get(vowel_letter)
    
    return vowel_class, speaker, gender


def detect_voice_activity(audio, sr=SAMPLE_RATE, 
                          window_ms=10, hop_ms=5, 
                          energy_threshold=0.01):
    """
    使用短时能量检测语音活动区域
    
    Args:
        audio: 音频数据
        sr: 采样率
        window_ms: 窗口大小（毫秒）
        hop_ms: 步长（毫秒）
        energy_threshold: 能量阈值（相对于最大能量的比例）
    
    Returns:
        start_sample: 语音开始的采样点
        end_sample: 语音结束的采样点
    """
    window_samples = int(sr * window_ms / 1000)
    hop_samples = int(sr * hop_ms / 1000)
    
    # 计算短时能量
    energies = []
    for i in range(0, len(audio) - window_samples, hop_samples):
        frame = audio[i:i + window_samples]
        energy = np.sum(frame ** 2) / window_samples
        energies.append(energy)
    
    if not energies:
        return 0, len(audio)
    
    energies = np.array(energies)
    max_energy = np.max(energies)
    
    if max_energy == 0:
        return 0, len(audio)
    
    # 归一化能量
    norm_energies = energies / max_energy
    
    # 找到超过阈值的区域
    active_frames = norm_energies > energy_threshold
    
    if not np.any(active_frames):
        return 0, len(audio)
    
    # 找到第一个和最后一个活动帧
    active_indices = np.where(active_frames)[0]
    first_active = active_indices[0]
    last_active = active_indices[-1]
    
    # 转换回采样点
    start_sample = first_active * hop_samples
    end_sample = min((last_active + 1) * hop_samples + window_samples, len(audio))
    
    return start_sample, end_sample


def preprocess_vowel_audio(audio, sr=SAMPLE_RATE):
    """
    预处理元音音频：
    1. 检测语音活动区域
    2. 截取有效部分
    3. 归一化音量
    
    Returns:
        processed_audio: 处理后的音频，如果无效返回 None
    """
    if audio is None or len(audio) == 0:
        return None
    
    # 检测语音活动
    start, end = detect_voice_activity(
        audio, sr,
        window_ms=PREPROCESS_CONFIG['onset_window_ms'],
        hop_ms=PREPROCESS_CONFIG['onset_hop_ms'],
        energy_threshold=PREPROCESS_CONFIG['energy_threshold']
    )
    
    # 截取有效区域
    active_audio = audio[start:end]
    
    # 检查最小时长
    min_samples = int(sr * PREPROCESS_CONFIG['min_vowel_duration'])
    if len(active_audio) < min_samples:
        return None
    
    # 限制最大时长
    max_samples = int(sr * PREPROCESS_CONFIG['max_vowel_duration'])
    if len(active_audio) > max_samples:
        # 取中间部分
        excess = len(active_audio) - max_samples
        start_offset = excess // 2
        active_audio = active_audio[start_offset:start_offset + max_samples]
    
    # 归一化音量
    max_val = np.max(np.abs(active_audio))
    if max_val > 0:
        active_audio = active_audio / max_val * 0.9
    
    return active_audio.astype(np.float32)


def load_vowel_dataset(data_dir):
    """
    加载并预处理所有元音样本
    
    Returns:
        vowel_bank: dict, {vowel_class: [{'audio': ..., 'speaker': ..., 'gender': ...}, ...]}
    """
    vowel_bank = defaultdict(list)
    
    # 查找所有 wav 文件
    wav_files = glob.glob(os.path.join(data_dir, '*.wav'))
    
    print(f"找到 {len(wav_files)} 个音频文件")
    
    # 统计
    stats = {
        'total': 0,
        'parsed': 0,
        'loaded': 0,
        'valid': 0,
        'by_vowel': defaultdict(int),
        'by_gender': defaultdict(int),
        'skipped_reasons': defaultdict(int),
    }
    
    for filepath in tqdm(wav_files, desc="加载音频"):
        stats['total'] += 1
        filename = os.path.basename(filepath)
        
        # 解析文件名
        vowel_class, speaker, gender = parse_vowel_filename(filename)
        
        if vowel_class is None:
            stats['skipped_reasons']['parse_failed'] += 1
            continue
        stats['parsed'] += 1
        
        # 加载音频
        audio = load_audio_file(filepath)
        if audio is None:
            stats['skipped_reasons']['load_failed'] += 1
            continue
        stats['loaded'] += 1
        
        # 预处理
        processed = preprocess_vowel_audio(audio)
        if processed is None:
            stats['skipped_reasons']['too_short'] += 1
            continue
        stats['valid'] += 1
        
        # 保存
        vowel_bank[vowel_class].append({
            'audio': processed,
            'speaker': speaker,
            'gender': gender,
            'filepath': filepath
        })
        
        stats['by_vowel'][vowel_class] += 1
        stats['by_gender'][gender] += 1
    
    # 打印统计
    print(f"\n处理统计:")
    print(f"  总文件数: {stats['total']}")
    print(f"  解析成功: {stats['parsed']}")
    print(f"  加载成功: {stats['loaded']}")
    print(f"  有效样本: {stats['valid']}")
    print(f"\n跳过原因:")
    for reason, count in stats['skipped_reasons'].items():
        print(f"  {reason}: {count}")
    print(f"\n性别分布:")
    for gender, count in stats['by_gender'].items():
        print(f"  {gender}: {count}")
    
    return vowel_bank


print("函数定义完成!")

In [ ]:
# 加载数据集
print("="*60)
print("加载 Dataset of Vowels")
print("="*60)

vowel_bank = load_vowel_dataset(VOWEL_DATA_DIR)

print("\n" + "="*60)
print("加载结果:")
print("-"*40)
total = 0
for vowel in VOWEL_CLASSES[:-1]:  # 排除 silence
    count = len(vowel_bank.get(vowel, []))
    print(f"  {vowel}: {count} 个有效样本")
    total += count
print("-"*40)
print(f"  总计: {total} 个有效样本")

In [ ]:
# 可视化一些样本，验证预处理效果
import matplotlib.pyplot as plt

print("预处理效果可视化")
print("="*60)

fig, axes = plt.subplots(5, 4, figsize=(16, 12))

for row_idx, vowel in enumerate(['A', 'E', 'I', 'O', 'U']):
    samples = vowel_bank.get(vowel, [])
    if not samples:
        continue
    
    # 随机选4个样本
    np.random.seed(42)
    indices = np.random.choice(len(samples), min(4, len(samples)), replace=False)
    
    for col_idx, idx in enumerate(indices):
        sample = samples[idx]
        audio = sample['audio']
        speaker = sample['speaker']
        
        ax = axes[row_idx, col_idx]
        
        # 绘制波形
        time_axis = np.arange(len(audio)) / SAMPLE_RATE * 1000  # 毫秒
        ax.plot(time_axis, audio, linewidth=0.5)
        ax.set_title(f"{vowel} - {speaker} ({len(audio)/SAMPLE_RATE*1000:.0f}ms)")
        ax.set_xlabel('Time (ms)')
        ax.set_ylabel('Amplitude')
        ax.set_ylim(-1, 1)
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'preprocessed_samples.png'), dpi=150)
plt.show()

# 统计处理后的音频长度分布
print("\n处理后音频长度分布:")
all_lengths = []
for vowel in VOWEL_CLASSES[:-1]:
    for sample in vowel_bank.get(vowel, []):
        all_lengths.append(len(sample['audio']) / SAMPLE_RATE * 1000)

if all_lengths:
    all_lengths = np.array(all_lengths)
    print(f"  最短: {np.min(all_lengths):.0f}ms")
    print(f"  最长: {np.max(all_lengths):.0f}ms")
    print(f"  平均: {np.mean(all_lengths):.0f}ms")
    print(f"  中位数: {np.median(all_lengths):.0f}ms")

## 5. 数据增强：元音拼接 + 噪声叠加

In [ ]:
def generate_noise(length, noise_type='white'):
    """
    生成噪声信号
    
    Args:
        length: 样本长度
        noise_type: 'white' 或 'pink'
    """
    if noise_type == 'white':
        return np.random.randn(length).astype(np.float32)
    elif noise_type == 'pink':
        # 简单的粉噪声生成（1/f 噪声）
        white = np.random.randn(length)
        fft = np.fft.rfft(white)
        freqs = np.fft.rfftfreq(length)
        freqs[0] = 1  # 避免除以零
        fft = fft / np.sqrt(freqs)
        pink = np.fft.irfft(fft, n=length)
        return pink.astype(np.float32)
    else:
        return np.zeros(length, dtype=np.float32)


def generate_silence_sample(length=INPUT_SAMPLES, noise_level=0.001):
    """
    生成静音样本（带轻微背景噪声）
    """
    noise_type = np.random.choice(['white', 'pink'])
    noise = generate_noise(length, noise_type)
    noise = noise / (np.max(np.abs(noise)) + 1e-8) * noise_level
    return noise


def get_random_vowel_segment(vowel_bank, vowel_class, min_len, max_len):
    """
    从元音库中随机获取一个指定长度的片段
    """
    samples = vowel_bank.get(vowel_class, [])
    if not samples:
        return None
    
    # 随机选择一个样本
    sample_info = np.random.choice(samples)
    audio = sample_info['audio'].copy()
    
    # 确定截取长度
    if len(audio) <= min_len:
        # 音频太短，直接使用全部
        segment = audio
    else:
        # 随机选择长度和起始位置
        actual_max = min(max_len, len(audio))
        if actual_max <= min_len:
            segment_len = len(audio)
        else:
            segment_len = np.random.randint(min_len, actual_max)
        
        max_start = len(audio) - segment_len
        start_idx = np.random.randint(0, max(1, max_start + 1))
        segment = audio[start_idx:start_idx + segment_len]
    
    return segment


def concatenate_vowels(vowel_bank, num_vowels=3, crossfade_samples=160):
    """
    随机选择并拼接多个元音
    
    Args:
        vowel_bank: 元音样本库
        num_vowels: 拼接的元音数量
        crossfade_samples: 交叉淡化的样本数（10ms @ 16kHz）
    
    Returns:
        concatenated: 拼接后的音频
        vowel_sequence: 元音序列标签 [(start, end, vowel_class), ...]
    """
    available_vowels = [v for v in VOWEL_CLASSES[:-1] if v in vowel_bank and len(vowel_bank[v]) > 0]
    
    if len(available_vowels) == 0:
        return None, None
    
    # 随机选择元音序列
    vowel_sequence = np.random.choice(available_vowels, num_vowels)
    
    segments = []
    labels = []
    current_pos = 0
    
    # 每个片段的长度范围
    min_len = INPUT_SAMPLES // 2  # 105ms
    max_len = INPUT_SAMPLES * 2   # 420ms
    
    for vowel in vowel_sequence:
        segment = get_random_vowel_segment(vowel_bank, vowel, min_len, max_len)
        
        if segment is None or len(segment) == 0:
            continue
        
        # 应用音量变化
        volume_factor = np.random.uniform(*AUGMENT_CONFIG['volume_range'])
        segment = segment * volume_factor
        
        # 记录标签
        labels.append({
            'start': current_pos,
            'end': current_pos + len(segment),
            'vowel': vowel
        })
        
        segments.append(segment)
        current_pos += len(segment) - crossfade_samples  # 重叠部分
    
    if len(segments) == 0:
        return None, None
    
    # 拼接（带交叉淡化）
    total_length = sum(len(s) for s in segments) - crossfade_samples * (len(segments) - 1)
    if total_length <= 0:
        return None, None
        
    concatenated = np.zeros(total_length, dtype=np.float32)
    
    pos = 0
    for i, segment in enumerate(segments):
        if i == 0:
            end_pos = min(pos + len(segment), total_length)
            concatenated[pos:end_pos] = segment[:end_pos - pos]
        else:
            # 交叉淡化
            fade_len = min(crossfade_samples, len(segment), total_length - pos)
            if fade_len > 0:
                fade_in = np.linspace(0, 1, fade_len)
                fade_out = np.linspace(1, 0, fade_len)
                
                # 淡出前一段的尾部
                concatenated[pos:pos + fade_len] *= fade_out
                # 叠加当前段（淡入开头）
                segment_copy = segment.copy()
                segment_copy[:fade_len] *= fade_in
                
                end_pos = min(pos + len(segment_copy), total_length)
                copy_len = end_pos - pos
                concatenated[pos:end_pos] += segment_copy[:copy_len]
        
        pos += len(segment) - crossfade_samples
    
    return concatenated, labels


def extract_training_sample(audio, labels, sample_length=INPUT_SAMPLES):
    """
    从拼接后的音频中提取训练样本
    
    Returns:
        sample: 音频片段
        label: 主要元音标签（占比最大的）
    """
    if len(audio) < sample_length:
        # 填充
        padded = np.zeros(sample_length, dtype=np.float32)
        start = (sample_length - len(audio)) // 2
        padded[start:start + len(audio)] = audio
        audio = padded
        # 调整标签位置
        for l in labels:
            l['start'] += start
            l['end'] += start
    
    # 随机选择起始位置
    max_start = len(audio) - sample_length
    if max_start <= 0:
        start_idx = 0
    else:
        start_idx = np.random.randint(0, max_start)
    
    end_idx = start_idx + sample_length
    sample = audio[start_idx:end_idx]
    
    # 确定主要标签（窗口内占比最大的元音）
    vowel_coverage = defaultdict(int)
    for l in labels:
        overlap_start = max(start_idx, l['start'])
        overlap_end = min(end_idx, l['end'])
        if overlap_end > overlap_start:
            vowel_coverage[l['vowel']] += overlap_end - overlap_start
    
    if vowel_coverage:
        label = max(vowel_coverage, key=vowel_coverage.get)
    else:
        label = 'silence'
    
    return sample, label


def add_noise(audio, noise_level):
    """
    添加背景噪声
    """
    noise_type = np.random.choice(['white', 'pink'])
    noise = generate_noise(len(audio), noise_type)
    noise = noise / (np.max(np.abs(noise)) + 1e-8) * noise_level
    
    return audio + noise


print("数据增强函数定义完成!")

In [ ]:
# 生成增强后的训练数据
print("="*60)
print("生成训练数据（拼接 + 噪声增强）")
print("="*60)

np.random.seed(42)

generated_samples = defaultdict(list)
samples_per_vowel = AUGMENT_CONFIG['samples_per_vowel']

# 为每个元音类别生成样本
for target_vowel in tqdm(VOWEL_CLASSES[:-1], desc="生成样本"):
    count = 0
    attempts = 0
    max_attempts = samples_per_vowel * 10
    
    while count < samples_per_vowel and attempts < max_attempts:
        attempts += 1
        
        # 随机拼接元音（确保包含目标元音）
        num_vowels = np.random.randint(
            AUGMENT_CONFIG['concat_min_vowels'],
            AUGMENT_CONFIG['concat_max_vowels'] + 1
        )
        
        concat_audio, labels = concatenate_vowels(vowel_bank, num_vowels)
        
        if concat_audio is None:
            continue
        
        # 提取样本
        sample, label = extract_training_sample(concat_audio, labels)
        
        # 只保留目标元音的样本（或允许一定比例的其他元音以增加多样性）
        if label != target_vowel and np.random.random() > 0.1:
            continue
        
        # 随机添加噪声
        if np.random.random() < AUGMENT_CONFIG['noise_prob']:
            noise_level = np.random.uniform(*AUGMENT_CONFIG['noise_level_range'])
            sample = add_noise(sample, noise_level)
        
        # 归一化
        max_val = np.max(np.abs(sample))
        if max_val > 0:
            sample = sample / max_val * 0.9
        
        generated_samples[label].append(sample)
        if label == target_vowel:
            count += 1

# 生成静音样本
print("\n生成静音样本...")
for _ in range(samples_per_vowel):
    silence = generate_silence_sample()
    generated_samples['silence'].append(silence)

# 统计
print("\n生成结果:")
print("-"*40)
total = 0
for vowel in VOWEL_CLASSES:
    count = len(generated_samples.get(vowel, []))
    print(f"  {vowel}: {count} 个样本")
    total += count
print("-"*40)
print(f"  总计: {total} 个样本")

## 6. 定义模型

与 TIMIT 版本相同的模型架构，包含 MelSpectrogram 层以确保训练和推理一致。

In [ ]:
import tensorflow as tf

def create_mel_filterbank(num_mels=N_MELS, num_spectrogram_bins=FFT_LENGTH // 2 + 1,
                          sample_rate=SAMPLE_RATE, lower_freq=0.0, upper_freq=8000.0):
    """创建 Mel 滤波器组矩阵"""
    return tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=num_mels,
        num_spectrogram_bins=num_spectrogram_bins,
        sample_rate=sample_rate,
        lower_edge_hertz=lower_freq,
        upper_edge_hertz=upper_freq
    ).numpy()


class MelSpectrogram(tf.keras.layers.Layer):
    """
    Mel 频谱提取层 - 将原始波形转换为 Log-Mel 特征
    该层会被导出到 TensorFlow.js，确保训练和推理一致
    """

    def __init__(self, frame_length=FRAME_LENGTH, frame_step=FRAME_STEP,
                 fft_length=FFT_LENGTH, num_mels=N_MELS,
                 sample_rate=SAMPLE_RATE, **kwargs):
        super().__init__(**kwargs)
        self.frame_length = frame_length
        self.frame_step = frame_step
        self.fft_length = fft_length
        self.num_mels = num_mels
        self.sample_rate = sample_rate

    def build(self, input_shape):
        mel_filterbank = create_mel_filterbank(
            num_mels=self.num_mels,
            num_spectrogram_bins=self.fft_length // 2 + 1,
            sample_rate=self.sample_rate
        )
        self.mel_filterbank = self.add_weight(
            name='mel_filterbank',
            shape=mel_filterbank.shape,
            initializer=tf.constant_initializer(mel_filterbank),
            trainable=False
        )
        super().build(input_shape)

    def call(self, waveform):
        # STFT
        stft = tf.signal.stft(
            waveform,
            frame_length=self.frame_length,
            frame_step=self.frame_step,
            fft_length=self.fft_length,
            window_fn=tf.signal.hann_window
        )

        # 幅度谱
        magnitude = tf.abs(stft)

        # Mel 滤波
        mel = tf.matmul(magnitude, self.mel_filterbank)

        # 对数压缩
        log_mel = tf.math.log(mel + 1e-6)

        # 添加通道维度 [batch, time, mels, 1]
        return tf.expand_dims(log_mel, -1)

    def get_config(self):
        config = super().get_config()
        config.update({
            'frame_length': self.frame_length,
            'frame_step': self.frame_step,
            'fft_length': self.fft_length,
            'num_mels': self.num_mels,
            'sample_rate': self.sample_rate,
        })
        return config


def create_vowel_model(include_mel_layer=True):
    """
    创建元音识别模型

    Args:
        include_mel_layer: 是否包含 Mel 特征提取层
            - True: 输入原始波形，用于最终导出
            - False: 输入预提取的 Mel 特征，用于加速训练
    """
    num_time_steps = int((INPUT_SAMPLES - FRAME_LENGTH) / FRAME_STEP) + 1

    if include_mel_layer:
        waveform_input = tf.keras.Input(shape=(INPUT_SAMPLES,), name='waveform')
        x = MelSpectrogram(name='mel_spectrogram')(waveform_input)
    else:
        x = tf.keras.Input(shape=(num_time_steps, N_MELS, 1), name='mel_input')
        waveform_input = x

    # CNN 分类部分
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1')(x if include_mel_layer else waveform_input)
    x = tf.keras.layers.BatchNormalization(name='bn1')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), name='pool1')(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2')(x)
    x = tf.keras.layers.BatchNormalization(name='bn2')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2), name='pool2')(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='conv3')(x)
    x = tf.keras.layers.BatchNormalization(name='bn3')(x)
    x = tf.keras.layers.GlobalAveragePooling2D(name='gap')(x)

    x = tf.keras.layers.Dense(32, activation='relu', name='dense1')(x)
    x = tf.keras.layers.Dropout(0.3, name='dropout')(x)

    output = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid', name='vowel_output')(x)

    model = tf.keras.Model(inputs=waveform_input, outputs=output, name='vowel_classifier')
    return model


def create_full_model_from_weights(feature_model):
    """从训练好的特征模型创建完整模型（包含 Mel 层）"""
    full_model = create_vowel_model(include_mel_layer=True)

    for layer in feature_model.layers:
        if layer.name in [l.name for l in full_model.layers]:
            if layer.name not in ['mel_spectrogram', 'waveform', 'mel_input']:
                full_model.get_layer(layer.name).set_weights(layer.get_weights())

    return full_model


# 测试模型创建
print("测试模型创建...")
test_model = create_vowel_model(include_mel_layer=True)
test_model.summary()

## 7. 准备训练数据集

In [ ]:
from sklearn.model_selection import train_test_split

def extract_mel_features(audio_batch):
    """提取 Mel 频谱特征"""
    stft = tf.signal.stft(
        audio_batch,
        frame_length=FRAME_LENGTH,
        frame_step=FRAME_STEP,
        fft_length=FFT_LENGTH,
        window_fn=tf.signal.hann_window
    )

    magnitude = tf.abs(stft)

    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=N_MELS,
        num_spectrogram_bins=FFT_LENGTH // 2 + 1,
        sample_rate=SAMPLE_RATE,
        lower_edge_hertz=0.0,
        upper_edge_hertz=8000.0
    )

    mel = tf.matmul(magnitude, mel_filterbank)
    log_mel = tf.math.log(mel + 1e-6)

    return tf.expand_dims(log_mel, -1).numpy()


# 准备数据
print("准备训练数据...")

all_audio = []
all_labels = []

for vowel_idx, vowel in enumerate(VOWEL_CLASSES):
    samples = generated_samples.get(vowel, [])
    for sample in samples:
        all_audio.append(sample)
        all_labels.append(vowel_idx)

all_audio = np.array(all_audio, dtype=np.float32)
all_labels = np.array(all_labels, dtype=np.int32)

print(f"总样本数: {len(all_audio)}")

# 划分训练/验证/测试集 (80/10/10)
train_audio, temp_audio, train_labels, temp_labels = train_test_split(
    all_audio, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

val_audio, test_audio, val_labels, test_labels = train_test_split(
    temp_audio, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(f"\n数据集划分:")
print(f"  训练集: {len(train_audio)} 样本")
print(f"  验证集: {len(val_audio)} 样本")
print(f"  测试集: {len(test_audio)} 样本")

# 提取 Mel 特征
print("\n提取 Mel 特征...")
EXTRACT_BATCH_SIZE = 512

def batched_mel_extraction(audio_data, batch_size):
    if len(audio_data) == 0:
        return np.array([])
    all_mels = []
    for i in range(0, len(audio_data), batch_size):
        batch_audio = audio_data[i:i + batch_size]
        batch_mel = extract_mel_features(batch_audio)
        all_mels.append(batch_mel)
    return np.concatenate(all_mels, axis=0)

train_mel = batched_mel_extraction(train_audio, EXTRACT_BATCH_SIZE)
val_mel = batched_mel_extraction(val_audio, EXTRACT_BATCH_SIZE)
test_mel = batched_mel_extraction(test_audio, EXTRACT_BATCH_SIZE)

print(f"\nMel 特征形状:")
print(f"  训练集: {train_mel.shape}")
print(f"  验证集: {val_mel.shape}")
print(f"  测试集: {test_mel.shape}")

## 8. 训练模型

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# 计算类别权重
class_weights = compute_class_weight(
    'balanced',
    classes=np.arange(NUM_CLASSES),
    y=train_labels
)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}
print(f"类别权重: {class_weight_dict}")

# 将标签转换为 one-hot 编码（sigmoid + binary_crossentropy 需要）
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, NUM_CLASSES)
val_labels_onehot = tf.keras.utils.to_categorical(val_labels, NUM_CLASSES)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, NUM_CLASSES)
print(f"标签形状: {train_labels_onehot.shape}")

# 创建模型
print("\n创建模型...")
model = create_vowel_model(include_mel_layer=False)
model.summary()

# 编译
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 回调
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=EARLY_STOPPING_PATIENCE,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(CHECKPOINT_DIR, 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

In [ ]:
# 开始训练
print("开始训练...")
history = model.fit(
    train_mel, train_labels_onehot,
    validation_data=(val_mel, val_labels_onehot),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

## 9. 评估与可视化

In [ ]:
import matplotlib.pyplot as plt

# 绘制训练历史
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history.history['accuracy'], label='Train')
axes[0].plot(history.history['val_accuracy'], label='Validation')
axes[0].set_title('Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history.history['loss'], label='Train')
axes[1].plot(history.history['val_loss'], label='Validation')
axes[1].set_title('Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history.png'), dpi=150)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# 评估测试集
print("评估测试集...")
test_loss, test_acc = model.evaluate(test_mel, test_labels_onehot, verbose=0)
print(f"测试集准确率: {test_acc:.4f}")
print(f"测试集损失: {test_loss:.4f}")

# 预测
predictions = model.predict(test_mel, verbose=0)
pred_labels = np.argmax(predictions, axis=1)

# 混淆矩阵
cm = confusion_matrix(test_labels, pred_labels)

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)

ax.set(
    xticks=np.arange(NUM_CLASSES),
    yticks=np.arange(NUM_CLASSES),
    xticklabels=VOWEL_CLASSES,
    yticklabels=VOWEL_CLASSES,
    title='Confusion Matrix',
    ylabel='True',
    xlabel='Predicted'
)

thresh = cm.max() / 2.
for i in range(NUM_CLASSES):
    for j in range(NUM_CLASSES):
        ax.text(j, i, format(cm[i, j], 'd'),
               ha="center", va="center",
               color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'confusion_matrix.png'), dpi=150)
plt.show()

# 分类报告
print("\n分类报告:")
print(classification_report(test_labels, pred_labels, target_names=VOWEL_CLASSES))

## 10. 导出模型

In [ ]:
# 创建完整模型（包含 Mel 层）
print("创建完整模型...")
full_model = create_full_model_from_weights(model)

# 测试
print("\n测试完整模型...")
test_input = test_audio[:5]
full_output = full_model.predict(test_input, verbose=0)

for i in range(5):
    true_label = VOWEL_CLASSES[test_labels[i]]
    pred_label = VOWEL_CLASSES[np.argmax(full_output[i])]
    conf = np.max(full_output[i])
    print(f"  样本 {i+1}: 真实={true_label}, 预测={pred_label} (置信度={conf:.3f})")

In [ ]:
# 保存模型
keras_path = os.path.join(OUTPUT_DIR, 'vowel_model.keras')
full_model.save(keras_path)
print(f"Keras 模型已保存: {keras_path}")

saved_model_path = os.path.join(OUTPUT_DIR, 'saved_model')
full_model.export(saved_model_path)
print(f"SavedModel 已导出: {saved_model_path}")

## 11. 转换为 TensorFlow.js

In [ ]:
import subprocess

tfjs_output_path = os.path.join(OUTPUT_DIR, 'tfjs_model')
os.makedirs(tfjs_output_path, exist_ok=True)

cmd = [
    'tensorflowjs_converter',
    '--input_format=tf_saved_model',
    '--output_format=tfjs_graph_model',
    '--quantize_uint8',
    '--skip_op_check',
    saved_model_path,
    tfjs_output_path
]

print("转换为 TensorFlow.js...")
result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode == 0:
    print("\n✅ 转换成功!")
    print("\n输出文件:")
    total_size = 0
    for f in os.listdir(tfjs_output_path):
        fpath = os.path.join(tfjs_output_path, f)
        size = os.path.getsize(fpath)
        total_size += size
        print(f"  {f}: {size / 1024:.1f} KB")
    print(f"\n总大小: {total_size / 1024:.1f} KB")
else:
    print(f"\n❌ 转换失败!")
    print(f"错误: {result.stderr}")

## 12. 下载模型

In [ ]:
import shutil
from google.colab import files

# 打包下载
zip_path = '/content/tfjs_model_hillenbrand.zip'
shutil.make_archive('/content/tfjs_model_hillenbrand', 'zip', tfjs_output_path)

print(f"模型已打包: {zip_path}")
print(f"大小: {os.path.getsize(zip_path) / 1024:.1f} KB")

files.download(zip_path)

In [ ]:
# 备份到 Google Drive
drive_output = '/content/drive/MyDrive/vowel_model_hillenbrand'
os.makedirs(drive_output, exist_ok=True)

shutil.copytree(OUTPUT_DIR, os.path.join(drive_output, 'output'), dirs_exist_ok=True)

print(f"\n✅ 所有输出已保存到 Google Drive:")
print(f"   {drive_output}")

---

## 🎉 完成!

使用 Dataset of Vowels 训练的模型应该在纯元音输入场景下表现更好。

### 与 TIMIT 版本的关键区别

| 特性 | TIMIT | Dataset of Vowels |
|------|-------|-------------------|
| 元音类型 | 嵌入在词中 (辅音-元音-辅音) | 孤立元音录音 |
| 数据预处理 | 无 | 能量检测 + 有效区域截取 + 音量归一化 |
| 数据增强 | 无 | 元音拼接 + 噪声叠加 |
| 适用场景 | 连续语音识别 | 纯元音/歌唱识别 |

### 下一步

1. 将下载的模型替换 `public/models/vowel/` 中的文件
2. 在 `/debug-ml` 页面测试真实录音效果
3. 如果效果仍不理想，可以尝试：
   - 调整 `PREPROCESS_CONFIG` 中的能量阈值
   - 调整 `AUGMENT_CONFIG` 参数
   - 增加更多噪声类型
   - 添加混响增强